# SQL Murder Mystery

<img src="./174092-clue-illustration.png" width="500" height="300" style="display: block; margin: 0 auto">

A crime has taken place and the detective needs your help. The detective gave
you the crime scene report, but you somehow lost it. You vaguely remember that
the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it
took place in ​SQL City​. Start by retrieving the corresponding crime scene
report from the police department’s database!

## Table schema

<img src="./schema.png" width="700" style="display: block; margin: 0 auto">

## Setup

In [3]:
%load_ext sql
%sql sqlite:///sql-murder-mystery.db

'Connected: @sql-murder-mystery.db'

In [6]:
%sql settings

 * sqlite:///sql-murder-mystery.db
(sqlite3.OperationalError) near "settings": syntax error
[SQL: settings]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


## Solving the case

From the first clue, we know the murder takes place in SQL City on 15 August
2018. Let's see if there's  any crime scene report that matches our criteria:

In [20]:
%%sql
select * from crime_scene_report csr where csr.type == 'murder' and city = 'SQL City' and date = '20180115'

 * sqlite:///sql-murder-mystery.db
Done.


date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


Nice! From the crime scene report, we've obtained some information about the 2
witnesses. Let's see if we can identify them.

In [24]:
%%sql
select *
from person
where name like '%Annabel%' and address_street_name = 'Franklin Ave'
union all
select *
from person where address_street_name = 'Northwestern Dr'
and address_number = (select max(address_number) from person where address_street_name = 'Northwestern Dr')

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
16371,Annabel Miller,490173,103,Franklin Ave,318771143
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


We're in luck! We've identified the 2 witnesses in the crime scene report. Let's
see what they said to the policy by looking at the `interview` table. Since the
interviews are identified by `person_id`, we need to join our witness
information to the `person` table, then finally to the `interview` table.

In [25]:
%%sql
with witnesses as (
    select *
    from person
    where name like '%Annabel%' and address_street_name = 'Franklin Ave'
    union all
    select *
    from person where address_street_name = 'Northwestern Dr'
    and address_number = (select max(address_number) from person where address_street_name = 'Northwestern Dr')
)
select w.name, i.transcript from witnesses w
join person p on w.ssn = p.ssn
join interview i on i.person_id = p.id

 * sqlite:///sql-murder-mystery.db
Done.


name,transcript
Annabel Miller,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."
Morty Schapiro,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [ ]:


/**
select * from get_fit_now_check_in ci
join get_fit_now_member m on ci.membership_id = m.id
join person p on m.person_id = p.id
join witnesses w on p.ssn = w.ssn
**/


